# 베르누이 나이브 베이즈 모델 (Bernoulli Naive Bayes)

<br>

## ✅베르누이 나이브 베이즈
- 데이터의 특징이 **0 또는 1로 표현** 됐을 때 사용
- e.g. 주사위를 10번 던졌을 때 1이 한 번, 2가 두 번, 3이 세 번, 4가 네 번 나왔을 경우, 주사위를 10번 던진 결과 데이터를 (1, 1, 1, 1, 0, 0)과 같이 나타낼 수 있음
    - 각 인덱스는 주사위의 면을 뜻함
    - 숫자가 출현했을 때는 1, 출현하지 않았을 때는 0으로 나타낸 것
- 데이터 출현 여부에 따라 1 또는 0으로 구분 됐을 때 사용
- e.g. 스팸 메일 분류

In [1]:
import numpy as np
import pandas as pd

# 베르누이 나이브 베이즈
from sklearn.naive_bayes import BernoulliNB
from sklearn.feature_extraction.text import CountVectorizer

# 성능평가
from sklearn import metrics
from sklearn.metrics import accuracy_score

<hr>

## 01. 문제 정의
##### *베르누이 나이브 베이즈 모델을 사용하여 스팸 메일을 분류*

<hr>

## 02. 데이터 수집
##### *이번 실습에서는 간단한 스팸 메일 분류 실습을 위해 아래 이메일 타이틀과 스팸 여부가 있는 데이터를 사용*

In [2]:
# train data
email_list = [
                {'email title': 'free game only today', 'spam': True},
                {'email title': 'cheapest flight deal', 'spam': True},
                {'email title': 'limited time offer only today only today', 'spam': True},
                {'email title': 'today meeting schedule', 'spam': False},
                {'email title': 'your flight schedule attached', 'spam': False},
                {'email title': 'your credit card statement', 'spam': False}
             ]

In [3]:
# test data
test_email_list = [
                {'email title': 'free flight offer', 'spam': True},
                {'email title': 'hey traveler free flight deal', 'spam': True},
                {'email title': 'limited free game offer', 'spam': True},
                {'email title': 'today flight schedule', 'spam': False},
                {'email title': 'your credit card attached', 'spam': False},
                {'email title': 'free credit card offer only today', 'spam': False}
                ]

In [4]:
# train df
train_df = pd.DataFrame(email_list)
test_df = pd.DataFrame(test_email_list)
train_df

,email title,spam
0,free game only today,True
1,cheapest flight deal,True
2,limited time offer only today only today,True
3,today meeting schedule,False
4,your flight schedule attached,False
5,your credit card statement,False


<hr>

## 03. 데이터 전처리
##### *sklearn의 베르누이 NB 분류기는 숫자만을 다루기 떄문에 DV 값 True와 False를 1과 0으로 치환* 

In [5]:
train_df['label'] = train_df['spam'].map({True:1, False:0})
train_df

,email title,spam,label
0,free game only today,True,1
1,cheapest flight deal,True,1
2,limited time offer only today only today,True,1
3,today meeting schedule,False,0
4,your flight schedule attached,False,0
5,your credit card statement,False,0


In [6]:
# 학습에 사용될 데이터(IV)와 분류값(DV)을 나눔
df_x = train_df['email title']
df_y = train_df['label']

In [7]:
# 클래스 객체 생성
cv = CountVectorizer(binary=True)     # 매개변수 binary: 
x_traincv = cv.fit_transform(df_x)    # fit()과 transform() 연산을 한 번에 처리하는 메소드

In [8]:
### `cv.fit_transform()` 함수 ###

# excel 파일에 df_x 데이터가 있다고 가정
# 1. 단어 하나하나를 꺼내와서 나열 (띄어쓰기 단위)
# 2. 이전에 있던 단어를 만나면 skip (중복되지 않도록)
# 3. 데이터에 있는 모든 단어를 가져오면 알파벳 순으로 정렬 (중복 단어 없이)  

# e.g. 정렬 결과 총 20개의 단어가 있다고 가정)

# 4. 단어의 갯수에 맞춰 20개의 공간 할당 & 각각의 단어에 인덱스 할당
# 5. 다시 email_title의 데이터를 하나씩 읽으면서 해당 단어가 있으면 1로 변환 >> 없으면 0 변환
# 6. df_x에는 총 6개의 행이 있으므로 6개의 공간이 할당됨  

### e.g.  
#  ... | cheapest | flight | ... | game | today  
#  ... |    0     |   0    | ... |  1   |   1  
#  >> 각각의 문장(row)마다 이런 형태로 변환됨

In [9]:
# 정렬된 단어 확인
print('정렬된 단어 확인:', cv.get_feature_names_out())
print('단어의 수:', len(cv.get_feature_names_out()))

정렬된 단어 확인: ['attached' 'card' 'cheapest' 'credit' 'deal' 'flight' 'free' 'game'
 'limited' 'meeting' 'offer' 'only' 'schedule' 'statement' 'time' 'today'
 'your']
단어의 수: 17


In [10]:
# 변환된 결과 확인
encoded_input = x_traincv.toarray()
encoded_input

# 5번째 값만 첫 번째 값에 1을 가지고 있으므로 'attached'라는 단어가 있을 것

array([[0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0],
       [0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1],
       [0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1]], dtype=int64)

In [11]:
# 0과 1로 변환된 encoded_input 값을 문자열로 변경
print(cv.inverse_transform(encoded_input[[0]]))           # 리스트 인덱싱으로 각각의 데이터에 접근
print(cv.inverse_transform(encoded_input[[2]]))

[array(['free', 'game', 'only', 'today'], dtype='<U9')]
[array(['limited', 'offer', 'only', 'time', 'today'], dtype='<U9')]


<hr>

## 04. 베르누이 나이브 베이즈 분류 모델

In [12]:
# 학습 데이터로 베르누이 분류기를 학습
bnb = BernoulliNB()
y_train = df_y.astype('int')            # 실수 >> 정수 형변환
bnb.fit(x_traincv, y_train)      # 지도학습모델이므로 정답 데이터도 줘야 함

BernoulliNB()

In [13]:
# 테스트 데이터로 성능 평가
test_df['label'] = test_df['spam'].map({True:1, False:0})
test_df

,email title,spam,label
0,free flight offer,True,1
1,hey traveler free flight deal,True,1
2,limited free game offer,True,1
3,today flight schedule,False,0
4,your credit card attached,False,0
5,free credit card offer only today,False,0


In [14]:
# 테스트 데이터 DV/IV 분류
test_x = test_df['email title']
test_y = test_df['label']

In [15]:
# test_x를 숫자로 변환할 때
# 입력(훈련) 사용한 변환기를 그대로 사용해야 함>> `cv`
# 해당 훈련 모델에 최적화된 형태로 변환해야 하므로 `fit()` 없이 just `transform()`만
# fit()은 내부적으로 학습을 하는 명령어, transform은 학습에 대한 변환

In [16]:
# x_traincv에 없는 단어들은 당연히 없음
x_testcv = cv.transform(test_x)

In [17]:
# 변환된 결과 확인
encoded_input = x_testcv.toarray()
encoded_input

array([[0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0],
       [1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0]], dtype=int64)

In [18]:
# 0과 1로 변환된 encoded_input 값을 문자열로 변경
cv.inverse_transform(encoded_input)

[array(['flight', 'free', 'offer'], dtype='<U9'),
 array(['deal', 'flight', 'free'], dtype='<U9'),
 array(['free', 'game', 'limited', 'offer'], dtype='<U9'),
 array(['flight', 'schedule', 'today'], dtype='<U9'),
 array(['attached', 'card', 'credit', 'your'], dtype='<U9'),
 array(['card', 'credit', 'free', 'offer', 'only', 'today'], dtype='<U9')]

<hr>

## 05. 성능평가

In [19]:
# 테스트 데이터로 모델의 정확도 평가
predicted = bnb.predict(x_testcv)
predicted

array([1, 1, 1, 0, 0, 1])

In [20]:
# 정확도
accuracy_score(test_y, predicted)

0.8333333333333334

In [21]:
print(metrics.classification_report(test_y, predicted))

              precision    recall  f1-score   support

           0       1.00      0.67      0.80         3
           1       0.75      1.00      0.86         3

    accuracy                           0.83         6
   macro avg       0.88      0.83      0.83         6
weighted avg       0.88      0.83      0.83         6



In [22]:
# 혼동행렬
metrics.confusion_matrix(test_y, predicted)

array([[2, 1],
       [0, 3]], dtype=int64)